# 데이터 경량화

## STEP 1. 전처리 함수 정의

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from tqdm import tqdm  # tqdm 라이브러리를 임포트하여 진행 상황을 시각화
import pandas as pd  # pandas 라이브러리를 데이터 처리를 위해 임포트
import numpy as np  # numpy 라이브러리를 수치 계산을 위해 임포트
import os  # os 라이브러리를 운영 체제와 상호 작용을 위해 임포트
import zipfile

def preprocess_file(csv_file, raw_path, raw_shuffle_data_path, divide_shuffles, label, processed_files_log):
    try:
        df = pd.read_csv(raw_path + csv_file, usecols=['drawing', 'key_id'])  # 원본 CSV 파일을 읽어 DataFrame으로 변환
        df['y'] = label  # 파일 순서에 따른 라벨을 할당

        df['cv'] = (df.key_id // 10000) % divide_shuffles  # key_id를 이용하여 데이터를 분할하기 위한 값을 계산

        for k in range(divide_shuffles):
            filename = raw_shuffle_data_path + f'train_k{k}.csv'  # 각 청크 파일의 이름을 정의
            chunk = df[df.cv == k]  # 현재 'cv' 값이 k인 데이터를 선택하여 새로운 DataFrame 생성
            chunk = chunk.drop(['key_id', 'cv'], axis=1)  # 'key_id'와 'cv' 열을 삭제

            if not os.path.exists(filename):
                chunk.to_csv(filename, index=False)  # 파일이 존재하지 않으면 새로 생성
            else:
                chunk.to_csv(filename, mode='a', header=False, index=False)  # 파일이 이미 있으면 데이터를 추가

        os.remove(raw_path + csv_file)  # 원본 CSV 파일 삭제

    except Exception as e:
        print(f"오류 발생: {csv_file} 처리 중 - {e}")  # 오류 발생 시 메시지 출력

    with open(processed_files_log, 'a') as log:
        log.write(csv_file + '\n')  # 처리된 파일 로그 기록


## STEP 2. 압축 해제 및 전처리

In [ ]:
def main():
    zip_path = '/content/drive/MyDrive/AIFFEL_QUESTs/quick_draw/quickdraw-doodle-recognition.zip'
    raw_path = "./data/train_raw/"
    raw_shuffle_data_path = raw_path + 'shuffle_raw_gzs/'
    divide_shuffles = 100
    processed_files_log = './data/processed_files.log'  # 로그 파일 경로

    # 디렉토리 생성
    if not os.path.exists(raw_shuffle_data_path):
        os.makedirs(raw_shuffle_data_path, exist_ok=True)

    # 로그 파일에서 이미 처리된 파일 목록 읽기
    if os.path.exists(processed_files_log):
        with open(processed_files_log, 'r') as log:
            processed_files = set(log.read().splitlines())
    else:
        processed_files = set()

    # 압축 파일 처리
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        file_list = [file for file in zip_ref.namelist() if file.startswith('train_raw/') and file.endswith('.csv')]

        # tqdm을 사용하여 진행 상황 시각화
        pbar = tqdm(total=len(file_list))
        for y, file in enumerate(file_list):
            csv_file = file.split('/')[-1]
            pbar.set_description(f"Processing {csv_file}")

            if csv_file not in processed_files:
                zip_ref.extract(file, './data/')
                preprocess_file(csv_file, raw_path, raw_shuffle_data_path, divide_shuffles, y, processed_files_log)
            pbar.update(1)
        pbar.close()

if __name__ == "__main__":
    main()

Processing cannon.csv:  17%|█▋        | 58/340 [21:55<1:58:40, 25.25s/it]